<a href="https://colab.research.google.com/github/Matheusbcy/-Data-Science-IA-/blob/main/Converse_com_documentos_usando_RAG_avan%C3%A7ada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit langchain
!pip install -q langchain_community langchain-huggingface langchain_ollama langchain_openai

In [2]:
!pip install -q faiss-cpu

In [ ]:
!pip install pypdf

In [ ]:
!pip install -q python-dotenv
!npm install -q localtunnel

In [5]:
!pip install -q langchain-groq

In [6]:
import faiss
import os
import getpass
from langchain_community.vectorstores import FAISS

In [7]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


In [12]:
%%writefile projeto3.py

import streamlit as st
import torch
import tempfile
import time
import os
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()

st.set_page_config(page_title="Converse com documentos", page_icon="📚")
st.title("Converse com documentos 📚")

# Criação de painel lateral na interface
uploads = st.sidebar.file_uploader(
    label="Enviar arquivos", type=["pdf"], accept_multiple_files=True
)

if not uploads:
    st.info("Por favor, envie algum arquivo para continuar")
    st.stop()

model_class = "gemma2-9b-it"  # moonshotai/kimi-k2-instruct

def model(model="gemma2-9b-it", temperature=0.6):
    llm = ChatGroq(
        model=model,
        max_tokens=1024,
        timeout=None,
        max_retries=2,
        temperature=temperature,
    )

    return llm


# Indexação e recuperação
def config_retriever(uploads):
    # Carregamento dos documentos

    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploads:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        loader = PyPDFLoader(temp_filepath)
        docs.extend(loader.load())

    # Divisão em pedaços de texto
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    # Embedding
    embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

    # Armazenamento
    vectorstore = FAISS.from_documents(splits, embeddings)
    vectorstore.save_local("vectorstore/db_faiss")

    # Configuração do Retriever
    retriever = vectorstore.as_retriever(
        search_type="mmr", search_kwargs={"k": 3, "fetch_k": 4}
    )

    return retriever



# Configuração da chain


def config_rag_chain(model_class, retriever):

    # Carregamento da LLM
    if model_class == "gemma2-9b-it":
        llm = model(model="gemma2-9b-it")
    elif model_class == "moonshotai/kimi-k2-instruct":
        llm = model(model="moonshotai/kimi-k2-instruct")

    token_s, token_e = "", ""

    # Prompt de contextualização
    context_q_system_prompt = "Give the following chat historyu and the follow-up question whitch might reference context in the chat history, formualte a standalone question which can be understood without the chat history. do NOT answer the question, just reformulate it if needed and otherwise return it as is"
    context_q_system_prompt = token_s + context_q_system_prompt
    context_q_user_prompt = "Question: {input}" + token_e
    context_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", context_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", context_q_user_prompt),
        ]
    )

    # Chain para contextualização
    history_aware_retriever = create_history_aware_retriever(
        llm=llm, retriever=retriever, prompt=context_q_prompt
    )

    # Prompt para perguntas e respostas
    qa_prompt_template = """Você é um assistente virtual prestativo e está respondendo perguntas gerais.
    Use os seguintes pedaços de contexto recuperado para responder a perguntas.
    Se você não sabe a resposta, apenas diga que não sabe. Mantenha a resposta concisa.
    Responda em português.  \n\n
    Pergunta: {input} \n
    Contexto: {context}
    """

    qa_prompt = PromptTemplate.from_template(token_s + qa_prompt_template + token_e)

    # Configurar LLM e Chain para perguntas e respostas
    qa_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

    return rag_chain

# Criando nova sessão
if "chat_history" not in st.session_state:
    st.session_state.chat_history = [
        AIMessage(content="Olá, sou o seu assistente virtual! Como posso ajuda você")
    ]

if "docs_list" not in st.session_state:
    st.session_state.docs_list = None

if "retriever" not in st.session_state:
    st.session_state.retriever = None

# Recuperando messagens
for message in st.session_state.chat_history:
    if isinstance(message, AIMessage):
        with st.chat_message("AI"):
            st.write(message.content)
    elif isinstance(message, HumanMessage):
        with st.chat_message("Human"):
            st.write(message.content)

start = time.time()
user_query = st.chat_input("Digite sua mensagem aqui...")

if user_query is not None and user_query != "" and uploads is not None:
    st.session_state.chat_history.append(HumanMessage(content = user_query))

    with st.chat_message("Human"):
        st.markdown(user_query)

    with st.chat_message("AI"):
        if st.session_state.docs_list != uploads:
            st.session_state.docs_list = uploads
            st.session_state.retriever = config_retriever(uploads)

        rag_chain = config_rag_chain(model_class, st.session_state.retriever)
        result = rag_chain.invoke({"input": user_query, "chat_history": st.session_state.chat_history})

        resp = result["answer"]
        st.write(resp)

        # Mostrar a fonte
        sources = result["context"]
        for idx, doc in enumerate(sources):
            source = doc.metadata["source"]
            file = os.path.basename(source)
            page = doc.metadata.get("page", "Página não especificada")

            ref = f":link: Fonte {idx}: *{file} - p. {page}*"
            with st.popover(ref):
                st.caption(doc.page_content)

    st.session_state.chat_history.append(AIMessage(content = resp))

end = time.time()
print("Tempo: ", end - start)

Overwriting projeto3.py


### Execução do Streamlit




In [13]:
!streamlit run projeto3.py &>/content/logs.txt &

In [ ]:
!wget -q -O - ipv4.icanhazip.com

!npx localtunnel --port 8501